# Assignment #1 Requirements 🎯

**Submission:**  
- Submit a Jupyter Notebook (`.ipynb`) file.

**Your Task:**  
- Train a model on your own dataset (e.g., ImageNet1K or FlowerDataset).

**Notebook must demonstrate:**
1. **Custom Model Declaration** 🛠️  
    - Define your own model class, including architecture initialization and forward pass.

2. **Checkpoint Loading** 💾  
    - Load saved model weights from a checkpoint file into your custom model instance.

3. **Dataset & Evaluation** 📊  
    - Load the specified dataset and perform model evaluation to demonstrate the trained model's performance.

## Submission Instructions 📥

You must submit the following files:

- **A trained Jupyter Notebook (.ipynb) file**  
  (The notebook should include all code and results demonstrating your training process.)

- **A model checkpoint file**  
  (Save your trained model weights using `torch.save()` or an equivalent method.)

Please ensure both files are included in your submission so that your training process and final model can be fully evaluated.

**Additional Notes:**  
- You can write your own jupyter notebook with brand-new code.
- Leave the model definition and dataset loading code as templates or with helpful comments if needed.
- After training, save a checkpoint, and include code to load the checkpoint and run inference, so that the grader can easily evaluate your model.

---

**Author:** Duhyeon Kim + Perplexity (GPT4.1)


### 1. Import libraries and set seed

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import random
import numpy as np
import os
from PIL import Image

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print("using device:", device)

using device: mps


### 2. Custom Dataset Class (Example)

In [3]:
class CustomDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        # Create list of data paths/labels
        self.samples = []
        self.class_to_idx = {}
        self.idx_to_class = {}

        # TODO: Collect class names and file paths according to folder structure
        classes = sorted(os.listdir(root))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        self.idx_to_class = {idx: cls_name for cls_name, idx in self.class_to_idx.items()}
        for cls_name in classes:
            cls_folder = os.path.join(root, cls_name)
            for fname in os.listdir(cls_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(cls_folder, fname)
                    self.samples.append((path, self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path)
        if self.transform:
            img = self.transform(img)
        return img, label


### 3. Prepare Dataset and DataLoader

In [10]:
# TODO: Change the paths below to your dataset paths
train_root = '../day2/cifar10_images/train'
test_root = '../day2/cifar10_images/test'

# TODO: Modify transforms as needed
train_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(...), # Add if needed
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = CustomDataset(train_root, transform=train_transform)
test_dataset = CustomDataset(test_root, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


### 4. Custom Model Declaration (Blank/Hint)

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        # TODO: Write your model architecture here
        # Example:
        # self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(16*16*16, 10)
        pass

    def forward(self, x):
        # TODO: Implement forward computation
        # Example:
        # x = self.pool(F.relu(self.conv1(x)))
        # x = x.view(-1, 16*16*16)
        # x = self.fc1(x)
        # return x
        pass

# Day1 Lecture Example Model:

# class CustomModel(nn.Module):
#     def __init__(self):
#         super(CustomModel, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(32 * 16 * 16, 128)
#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

### 5. Training Loop (Example)

In [12]:
model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

num_epochs = 10  # TODO: Change if needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 2.2499
Epoch [2/10], Loss: 2.1484
Epoch [3/10], Loss: 2.0525
Epoch [4/10], Loss: 1.9817
Epoch [5/10], Loss: 1.9367
Epoch [6/10], Loss: 1.9053
Epoch [7/10], Loss: 1.8820
Epoch [8/10], Loss: 1.8617
Epoch [9/10], Loss: 1.8441
Epoch [10/10], Loss: 1.8250


### 6. Save Checkpoint, Load Checkpoint, and Evaluate Model

In [13]:
torch.save(model.state_dict(), 'custom_model_ckpt.pth')
print("Checkpoint saved!")

# For submission: The evaluator should be able to run inference with this code
model_loaded = CustomModel().to(device)
model_loaded.load_state_dict(torch.load('custom_model_ckpt.pth', map_location=device))
model_loaded.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_loaded(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")


Checkpoint saved!
Test Accuracy: 36.58%
